#### Imports

In [ ]:
import pandas as pd
import numpy as np

from scipy.signal import butter, filtfilt, freqz
import matplotlib.pyplot as plt

from typing import Callable

#### Daten einlsene und als Dict [Sensorname:str, DataFrame] bereitstellen

In [ ]:
#json zu DF
df = pd.read_json("data/testdaten.json")
df

In [ ]:
df_non_Meta = df[1:].copy()
df_non_Meta.dropna(axis=1, how="all",inplace=True)

df_meta = df[0:1].copy()
df_meta.dropna(axis=1, inplace=True)

In [ ]:
df_meta.T

In [ ]:
df_non_Meta.columns

In [ ]:
df["sensor"].unique()

In [ ]:
#Dict mit key=Sensor, Value=DF(Values des Sensors) -> Alle NAN-Spalten löschen -> Index zurücksetzen
sensor_dfs = {sensor: grouped_dfs.dropna(axis=1, how="all").reset_index(drop=True) for sensor, grouped_dfs in df.groupby("sensor")}

# Metadaten aus den Senoren nehmen
if 'Metadata' in sensor_dfs:
    del sensor_dfs['Metadata']

In [ ]:
#Liste der Sensoren
sensor_list = list(sensor_dfs.keys())
sensor_list

In [ ]:
sensor_dfs["Accelerometer"]

In [ ]:
df_raw = sensor_dfs["Accelerometer"]
plt.figure(figsize=(10,4))
plt.plot(df_raw.index, df_raw["x"], label="x")
plt.plot(df_raw.index, df_raw["y"], label="y")
plt.plot(df_raw.index, df_raw["z"], label="z")
plt.title("Accelerometer Rohdaten (x,y,z)")
plt.xlabel("Index (Zeit oder Sample)")
plt.ylabel("Beschleunigung")
plt.grid(True, linestyle="--", linewidth=0.5)
plt.legend()
plt.show()

# ==> kein gravitation im Sonsorbild

In [ ]:
for key in sensor_dfs:
    print(f"{key} {sensor_dfs[key].shape}")
    # print(type(key))

#### Sensorabtastraten "schätzen" (AI based check):


In [ ]:

def estimate_rates_from_elapsed(sensor_dfs, col="seconds_elapsed"):
    results = []
    for sensor, df_s in sensor_dfs.items():
        if col not in df_s.columns or len(df_s) < 2:
            continue

        # Zeitdifferenzen in Sekunden
        dt = df_s[col].diff().dropna()
        dt = dt[dt > 0]

        if dt.empty:
            continue

        median_dt = dt.median()
        mean_dt = dt.mean()
        std_dt = dt.std(ddof=1)

        results.append({
            "sensor": sensor,
            "n_samples": len(df_s),
            "median_dt_ms": median_dt * 1000,   # s → ms
            "mean_dt_ms": mean_dt * 1000,
            "std_dt_ms": std_dt * 1000,
            "approx_rate_hz": 1.0 / median_dt   # Hz ≈ 1 / median(s)
        })

    return pd.DataFrame(results).sort_values("approx_rate_hz", ascending=False).reset_index(drop=True)




In [ ]:
rates_df = estimate_rates_from_elapsed(sensor_dfs, col="seconds_elapsed")
rates_df


#### Sensoren wählen, Prüfen und Time als Index setzen:

In [ ]:
def select_and_check_sensors(sensor_dfs: dict[str, pd.DataFrame], sensor_list: list[str]) -> dict[str, pd.DataFrame]:
    selected_sensors = {sensor: data for sensor, data in sensor_dfs.items() if sensor in sensor_list}
    for s, df in selected_sensors.items():
        print(f"{s:<20} {len(df):>10} Messpunkte{df.isna().sum().sum():>10} NaNs")

    return selected_sensors



In [ ]:
##### Das sind die ausgewählten Sensoren
SENSOR_LIST = ["Accelerometer","Gyroscope","GameOrientation","Location"]
selected_sensors = select_and_check_sensors(sensor_dfs, SENSOR_LIST)

In [ ]:
for k, v in sensor_dfs.items():
    if "time" not in v.columns:
        print(f"Spalte 'time' ist NICHT vorhanen in {k}")
    if "seconds_elapsed" not in v.columns:
        print(f"Spalte 'seconds_elapsed' ist NICHT vorhanen in {k}")

# --> alle Sonsoren enthalten beide Zeitspalten

In [ ]:
for k, v in sensor_dfs.items():
    if "time" in v.columns:
        print(f"-----------------------\n{k}\n{v["time"].head(3)}")

pd.to_datetime(1.755603e18, unit="ns", utc=True)
# --> UNIX time in Nanosekunden!

In [ ]:
# ALLGEMEINE Funktion um spätere Funktionen auf alle Sensoren anzuwenden:
def apply_to_all_sensors(func:Callable[[pd.DataFrame], pd.DataFrame], sensor_dfs:dict[str, pd.DataFrame]) -> dict[str, pd.DataFrame]:
    return {name: func(df) for name, df in sensor_dfs.items()}

In [ ]:
def time_to_index(df: pd.DataFrame) -> pd.DataFrame:
    df_timeindex = df.copy()
    df_timeindex.set_index(pd.to_datetime(df_timeindex["time"], unit="ns", utc=True),inplace=True)
    df_timeindex.index.name = "time_utc"
    df_timeindex.drop(columns="time", inplace=True)
    # später auf richtige reihenfolge der Zeitstempel, Duplikate und Lücken prüfen
    return df_timeindex



In [ ]:
#Datetime Index für alle Sensor DFs
selected_sensors = apply_to_all_sensors(time_to_index,selected_sensors)

In [ ]:
selected_sensors["Accelerometer"]

#### Sensorabtastraten "schätzen",  detailierter (AI based check):


In [ ]:
def timing_benchmark(df: pd.DataFrame, high_gap_factor: float = 3.0) -> dict:
    """
    MVP: Qualität der Zeitachse eines einzelnen Sensor-DFs beurteilen.
    Erwartet: DatetimeIndex (UTC, ns). Keine Änderungen am Signal.
    
    Kennzahlen:
      - rows, duration_s
      - n_deltas (Anzahl Abstände), n_nonpos (Δt <= 0)
      - median_dt_ns, mean_dt_ns, std_dt_ns, p95_dt_ns, max_dt_ns
      - approx_hz (1e9 / median_dt_ns)
      - jitter_cv = std_dt_ns / median_dt_ns
      - large_gap_count (Δt > high_gap_factor * median_dt_ns)
      - large_gap_max_ns (größte „große Lücke“)
    """
    out = {
        "rows": len(df),
        "duration_s": np.nan,
        "n_deltas": 0,
        "n_nonpos": 0,
        "median_dt_ns": np.nan,
        "mean_dt_ns": np.nan,
        "std_dt_ns": np.nan,
        "p95_dt_ns": np.nan,
        "max_dt_ns": np.nan,
        "approx_hz": np.nan,
        "jitter_cv": np.nan,
        "large_gap_factor": high_gap_factor,
        "large_gap_count": 0,
        "large_gap_max_ns": np.nan,
    }
    if len(df) < 2:
        return out

    # Gesamtdauer
    out["duration_s"] = (df.index[-1] - df.index[0]).total_seconds()

    # Δt in ns (Index ist datetime64[ns])
    ts_ns = df.index.view("int64")
    dt = np.diff(ts_ns)
    out["n_deltas"] = int(dt.size)
    if dt.size == 0:
        return out

    # nicht-positive Deltas nur für die Statistik zählen, für Kennzahlen ignorieren
    out["n_nonpos"] = int((dt <= 0).sum())
    dt_pos = dt[dt > 0]
    if dt_pos.size == 0:
        return out

    # robuste „typische Schrittweite“
    median_dt_ns = float(np.median(dt_pos))
    mean_dt_ns   = float(np.mean(dt_pos))
    std_dt_ns    = float(np.std(dt_pos, ddof=1)) if dt_pos.size > 1 else 0.0
    p95_dt_ns    = float(np.quantile(dt_pos, 0.95))
    max_dt_ns    = float(np.max(dt_pos))

    out["median_dt_ns"] = median_dt_ns
    out["mean_dt_ns"]   = mean_dt_ns
    out["std_dt_ns"]    = std_dt_ns
    out["p95_dt_ns"]    = p95_dt_ns
    out["max_dt_ns"]    = max_dt_ns
    out["approx_hz"]    = (1e9 / median_dt_ns) if median_dt_ns > 0 else np.nan
    out["jitter_cv"]    = (std_dt_ns / median_dt_ns) if median_dt_ns > 0 else np.nan

    # große Lücken gegenüber „typischem“ Schritt
    thr = high_gap_factor * median_dt_ns
    large = dt_pos[dt_pos > thr]
    out["large_gap_count"]  = int(large.size)
    out["large_gap_max_ns"] = float(np.max(large)) if large.size else np.nan
    return out


def timing_benchmark_all(sensors: dict[str, pd.DataFrame], high_gap_factor: float = 3.0) -> pd.DataFrame:
    rows = []
    for name, df in sensors.items():
        m = timing_benchmark(df, high_gap_factor=high_gap_factor)
        m["sensor"] = name
        rows.append(m)
    cols = ["sensor","rows","duration_s","n_deltas","n_nonpos",
            "median_dt_ns","mean_dt_ns","std_dt_ns","p95_dt_ns","max_dt_ns",
            "approx_hz","jitter_cv","large_gap_factor","large_gap_count","large_gap_max_ns"]
    return pd.DataFrame(rows)[cols].sort_values("sensor").reset_index(drop=True)


In [ ]:
bench = timing_benchmark_all(selected_sensors, high_gap_factor=3.0)
bench  # im Notebook als Tabelle anzeigen lassen

#### Auf feste Zeiten resamplen:

In [ ]:
STEPS = "5ms" #200hz

def resample_imu_sensors(df: pd.DataFrame, step: str = STEPS) -> pd.DataFrame:
    #out = df.resample(step).mean(numeric_only=True)
    out = df.resample(step, origin="epoch", label="left", closed="left").mean(numeric_only=True)
    out = out.interpolate(method="time", limit_direction="both")
    return out

def resample_location(df: pd.DataFrame, step: str = STEPS) -> pd.DataFrame:
    # resample speziell für die langsame abtastrate der Location
    out = df.resample(step, origin="epoch", label="left", closed="left").ffill()
    return out

def resample_selected_sensors(selected: dict[str, pd.DataFrame], step: str = STEPS) -> dict[str, pd.DataFrame]:
    out: dict[str, pd.DataFrame] = {}
    for name, df in selected.items():
        if name == "Location":
            out[name] = resample_location(df, step)
        else:
            out[name] = resample_imu_sensors(df, step)

    # Quick-Sanity: haben alle denselben Index?
    if not out:
        return out
    idx0 = next(iter(out.values())).index
    for n, d in out.items():
        if not d.index.equals(idx0):
            # Für MVP nicht hart abbrechen, nur sichtbar machen:
            print(f"⚠️ Index von {n} weicht ab (Länge={len(d)}, Start={d.index[0]}, Ende={d.index[-1]})")
            
    return out



In [ ]:
selected_sensors.keys()

In [ ]:
resampled_sensors = resample_selected_sensors(selected_sensors,STEPS)
#### NEXT:
#### Alle Senoren auf die länge vom ersten bis letzen GPS-fix aus "Locations" schneiden...  

In [ ]:
resampled_sensors["Location"].columns

#### Alle Sonsoren auf gemeinsamen (Location) Index slicen:
Für den MVP werden die Start und Endzeitpunkte auf den ersten und letzten GPS-Fix gesetzt, später muss das etwas intelligenter Implementiert werden.

In [ ]:
def location_window_no_nans(location_df: pd.DataFrame) -> tuple[pd.Timestamp, pd.Timestamp]:
    """
    Gibt (t0, t1) zurück:
      t0 = erster Index, dessen gesamte Zeile keine NaNs enthält
      t1 = letzter Index, dessen gesamte Zeile keine NaNs enthält

    Erwartet: Location hat bereits DatetimeIndex (z. B. nach time_to_index + Resampling).

    Hinweise:
    - MVP: keine Rücksicht auf Accuracy-Werte (horizontal/verticalAccuracy, etc.).
      Ein strengerer Fix könnte später z.B. eine Mindestgenauigkeit erfordern.
    - Gibt zusätzlich Info aus, wie viele Sekunden vorne/hinten abgeschnitten werden.
    """
    if len(location_df) == 0:
        raise ValueError("Location ist leer.")

    # Falls eine keine Zeilen ohne NaNs geben sollte
    valid = location_df.notna().all(axis=1)
    if not valid.any():
        raise ValueError("Keine vollständig NaN-freie Zeile in Location gefunden.")

    t0 = valid.idxmax()           # erster True-Index
    t1 = valid[::-1].idxmax()     # letzter True-Index

    # Debug-Infos: abgeschnittene Zeiträume
    total = (location_df.index[-1] - location_df.index[0]).total_seconds()
    cut_front = (t0 - location_df.index[0]).total_seconds()
    cut_back = (location_df.index[-1] - t1).total_seconds()
    print(f"⏱️ GPS-Window: Gesamt {total:.2f} s, vorne abgeschnitten {cut_front:.2f} s, hinten {cut_back:.2f} s")

    #TypeCheck t0, t1
    print("\n")
    print("Type of returned values:")
    print(type(t0))
    print(type(t1))
    

    return t0, t1

In [ ]:
start, end = location_window_no_nans(resampled_sensors["Location"])

In [ ]:
def slice_all_sensors(sensors:dict[str, pd.DataFrame], start:pd.Timestamp, end:pd.Timestamp) -> dict[str, pd.DataFrame]:
    sliced_sensors = {name : df.loc[start:end] for name, df in sensors.items()}
    return sliced_sensors

In [ ]:
sliced_sensors = slice_all_sensors(resampled_sensors, start, end)

for k, v in sliced_sensors.items():
    print(f"{k:<16} {len(v):>6} rows  Start={v.index[0]}  Ende={v.index[-1]}")

### Pre-Preprocessing HP-Filter für x,y,z der Sensoren

In [ ]:
def apply_highpass_axes(df: pd.DataFrame, fs=200.0, fc=0.5, order=4, suffix="_hp"):
    """
    Minimalversion: Butterworth-Highpass pro Achse (x,y,z).
    Hängt neue Spalten x_hp, y_hp, z_hp an.
    """
    nyq = fs / 2.0
    Wn = fc / nyq
    b, a = butter(order, Wn, btype="highpass")

    out = df.copy()
    for ax in ["x", "y", "z"]:
        if ax in out.columns:
            # TODO: später NaN-Handling einbauen
            # TODO: später prüfen ob Serie lang genug ist
            s = out[ax].astype(float)
            s_hp = filtfilt(b, a, s.to_numpy())
            out[ax + suffix] = s_hp
        else:
            out[ax + suffix] = np.nan  # TODO: später schöner handeln
    return out

In [ ]:
sliced_sensors["Accelerometer"] = apply_highpass_axes(sliced_sensors["Accelerometer"])
sliced_sensors["Gyroscope"] = apply_highpass_axes(sliced_sensors["Gyroscope"])
sliced_sensors["GameOrientation"] = apply_highpass_axes(sliced_sensors["GameOrientation"])
sliced_sensors["Location"] = apply_highpass_axes(sliced_sensors["Location"])

In [ ]:
sliced_sensors["Accelerometer"]

In [ ]:
#Visualisierung der Filterwirkung

# Zugriff direkt auf den Accelerometer-DF im Dict
df = sliced_sensors["Accelerometer"]

# wieviel Zeit anzeigen?
PLOT_SECONDS = 20
t0 = df.index[0]
t1 = t0 + pd.Timedelta(seconds=PLOT_SECONDS)
view = df.loc[t0:t1]

print(f"Zeige von {t0} bis {t1} | {len(view)} Punkte (~{PLOT_SECONDS} s)")

for ax in ["x","y","z"]:
    hp_col = f"{ax}_hp"
    if ax not in view.columns or hp_col not in view.columns:
        print(f"Überspringe {ax}: Spalte fehlt ({ax} / {hp_col})")
        continue

    plt.figure(figsize=(15,3))
    plt.plot(view.index, view[ax],     label=f"{ax} (raw)")
    plt.plot(view.index, view[hp_col], label=f"{hp_col} (high-pass)")
    plt.title(f"Accelerometer – {ax} vs. {hp_col}")
    plt.xlabel("Zeit")
    plt.ylabel("Beschleunigung [m/s²]")
    plt.grid(True, linestyle="--", linewidth=0.5)
    plt.legend()
    plt.tight_layout()
    plt.show()

In [ ]:
df = sliced_sensors["Accelerometer"]
print("Mittelwerte (ruhig liegend):")
print(df[["x","y","z"]].mean())
print("Magnitude (ruhig liegend):", np.sqrt((df[["x","y","z"]]**2).sum(axis=1)).mean())



In [ ]:
print("Mittelwerte (hochpassgefiltert):")
print(df[["x_hp","y_hp","z_hp"]].mean())
print("Magnitude (hochpassgefiltert):", np.sqrt((df[["x_hp","y_hp","z_hp"]]**2).sum(axis=1)).mean())

### Features für die jeweiligen Sensoren erstellen:

#### Dataframe mit variablen Zeitfenstern für die Features:

In [ ]:
### Globale Parameter:

WINDOW_LEN = 2.0 #Intervalllänge in Sekunden
WINDOW_STEP = 1.0 #Steps je Intervall in Sekunden

L = pd.to_timedelta(WINDOW_LEN,  unit="s")
S = pd.to_timedelta(WINDOW_STEP, unit="s")
# z.b. window_len = 2 und window_steps = 1 --> jede Sekunde ein 2 Sekunden Fenster =  50% Überlappung


In [ ]:
# 1) Referenzindex der Sensoren
ref_df = next(iter(sliced_sensors.values())) # Index des ersten Sonsors, da alle Sensoren auf den gleichen Index gesliced sind.
idx = ref_df.index
print("Referenzindex:", type(idx), "Länge:", len(idx))
print("Start:", idx[0], "Ende:", idx[-1])

start = idx[0]
end   = idx[-1]

In [ ]:
# --- Anzahl Fenster: n = floor((T - L)/S) + 1  (nur vollständige Fenster) ---
T = end - start
n = int(np.floor((T - L) / S)) + 1 if T >= L else 0

# --- Fensterstarts und -mitten (simpel: Mitte = Start + L/2) ---
if n > 0:
    t_start = start + np.arange(n) * S
    t_mid   = t_start + L/2
    windows = pd.DataFrame({"t_start": t_start, "t_mid": t_mid})
else:
    windows = pd.DataFrame({"t_start": pd.to_datetime([]), "t_mid": pd.to_datetime([])})

print(f"Fenster gebaut: {len(windows)} | Start={start} | Ende={end} | L={L} | S={S}")

windows # <---- Zeitfenster als Index für die Features 

#### Accelerometer --> Magnitude RMS (GRAVITATION IST SCHON RAUS --> NOCHMAL ÜBERARBEITEN!!!)

In [ ]:
sensor_name = "Accelerometer"
accel_df = sliced_sensors[sensor_name]

rows = []
for t0, tmid in zip(windows["t_start"], windows["t_mid"]):
    # Slice des Sensor-DF für dieses Fenster (rechts offen)
    mask  = (accel_df.index >= t0) & (accel_df.index < t0 + L)
    chunk = accel_df.loc[mask]
    if chunk.empty:
        raise RuntimeError(f"Leeres Fenster bei t0={t0} (Index {accel_df.index[0]} ... {accel_df.index[-1]})")
    # ohne Gravitation
    mag2 = chunk["x"]**2 + chunk["y"]**2 + chunk["z"]**2
    mag_rms = float(np.sqrt(np.mean(mag2)))
    # mit HP-Filter
    mag2_hp = chunk["x_hp"]**2 + chunk["y_hp"]**2 + chunk["z_hp"]**2
    mag_rms_hp = float(np.sqrt(np.mean(mag2_hp)))
    
    rows.append({"t_start": t0, "t_mid": tmid, "mag_rms": mag_rms, "mag_rms_hp": mag_rms_hp})

accelerometer_features = pd.DataFrame(rows).set_index("t_start")

In [ ]:
accelerometer_features

In [ ]:
# Visualisierung: mag_rms vs mag_rms_hp
plt.figure(figsize=(15, 6))

plt.subplot(2, 1, 1)
plt.plot(accelerometer_features["t_mid"], accelerometer_features["mag_rms"], 
         label="mag_rms (alle Frequenzen)", color="blue", linewidth=1.5)
plt.plot(accelerometer_features["t_mid"], accelerometer_features["mag_rms_hp"], 
         label="mag_rms_hp (langsame Bewegungen entfernt)", color="red", linewidth=1.5)
plt.title("Accelerometer Magnitude RMS Vergleich")
plt.xlabel("Zeit")
plt.ylabel("Magnitude RMS [m/s²]")
plt.legend()
plt.grid(True, linestyle="--", linewidth=0.5)

plt.subplot(2, 1, 2)
plt.plot(accelerometer_features["t_mid"], accelerometer_features["mag_rms"] - accelerometer_features["mag_rms_hp"], 
         label="Differenz (entfernte langsame Bewegungen)", color="green", linewidth=1.5)
plt.title("Durch HP-Filter entfernte langsame Bewegungskomponenten")
plt.xlabel("Zeit")
plt.ylabel("Differenz [m/s²]")
plt.legend()
plt.grid(True, linestyle="--", linewidth=0.5)

plt.tight_layout()
plt.show()

# Statistiken ausgeben
print(f"mag_rms (alle Frequenzen):        Mean={accelerometer_features['mag_rms'].mean():.4f}, Std={accelerometer_features['mag_rms'].std():.4f}")
print(f"mag_rms_hp (HP-gefiltert):        Mean={accelerometer_features['mag_rms_hp'].mean():.4f}, Std={accelerometer_features['mag_rms_hp'].std():.4f}")
print(f"Entfernte langsame Komponenten:   Mean={(accelerometer_features['mag_rms'] - accelerometer_features['mag_rms_hp']).mean():.4f}")